In [1]:
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
#loading data
df = pd.read_csv('LOR.csv', usecols = ['Age', 'EstimatedSalary', 'Purchased'])

x = df.iloc[:,0:2].values
y = df['Purchased'].values

In [3]:
#splitting data into train and test
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [4]:
#scaling data into some short range to make calculations easier
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
test_x = sc.fit_transform(test_x)

In [5]:
#initializing weights
w10, w11, w12, w13, w14, w15, w16, w17, w18, w20, w21, w22, w23 = [2 * random.random() - 1 for i in range(13)]

#initializing bias
x0 = 1
a10 = 1

# initializing learning rate
l = 0.05

#length of data
n = len(train_x)

In [6]:
#weights 1st layer
weights_1 = np.array([[w10, w11, w12],
               [w13, w14, w15],
               [w16, w17, w18]])

def sigmoid(val):
    return np.exp(val)/(1 + np.exp(val))

def sigmoid_derivative(val):
    outcome = sigmoid(val) * (1 - sigmoid(val))
    return outcome

def calculate_loss(pred, actual):
    loss = 1/n * sum((actual - pred) **2)
    return loss

i = 0
#specify the epoch number
epochs = 5000

while i < epochs:
    pred_result, sigmoid_derivatives_z20, sigmoid_derivatives_a20 = ([], [], [])
    a11_list, a12_list, a13_list, w20_list, w21_list, w22_list, w23_list, x1_list, x2_list = ([], [], [], [], [], [], [], [], [])
    
    for x1, x2 in train_x:
        #input layer
        inputs = np.array([[x0],[x1],[x2]])
        
        out1 = np.dot(weights_1, inputs)

        z11 = out1[0,0]
        z12 = out1[1,0]
        z13 = out1[2,0]

        # activation relu
        a11 = np.maximum(0, z11)
        a12 = np.maximum(0, z12)
        a13 = np.maximum(0, z13)

        #hidden layer 1st
        hidden_1 = np.array([a10, a11, a12, a13])
        
        #weights 2nd layer
        weights_2 = np.array([[w20],[w21],[w22],[w23]])

        out2 = np.dot(hidden_1, weights_2)
        z20 = out2[0]
        sigmoid_derivatives_z20.append(sigmoid_derivative(z20))
        a11_list.append(a11)
        a12_list.append(a12)
        a13_list.append(a13)
        w20_list.append(w20)
        w21_list.append(w21)
        w22_list.append(w22)
        w23_list.append(w23)
        x1_list.append(x1)
        x2_list.append(x2)
        
        #activation sigmoid
        a20 = sigmoid(z20)

        pred_result.append(a20)

    pred_result = np.array(pred_result)
    
    #calculating loss
    loss = calculate_loss(pred_result, train_y)
    
    #converting them into array
    a11_list, a12_list, a13_list = (np.array(a11_list), np.array(a12_list), np.array(a13_list))
    w20_list, w21_list, w22_list = (np.array(w20_list), np.array(w21_list), np.array(w22_list)) 
    w23_list, x1_list, x2_list = (np.array(w23_list), np.array(x1_list), np.array(x2_list))
    
    common_calc = (train_y - pred_result) * sigmoid_derivatives_z20
    #updating weights
    w20 = w20 - l * ((-2/n) * sum(common_calc))
    w21 = w21 - l * ((-2/n) * sum(common_calc * a11_list))
    w22 = w22 - l * ((-2/n) * sum(common_calc * a12_list))
    w23 = w23 - l * ((-2/n) * sum(common_calc * a13_list))
    
    #If after applying relu we got 0 then no need to update
    if a11:
        w10 = w10 - l * ((-2/n) * sum(common_calc * w21_list))
        w11 = w11 - l * ((-2/n) * sum(common_calc * w21_list * x1_list))
        w12 = w12 - l * ((-2/n) * sum(common_calc * w21_list * x2_list))
    if a12:
        w13 = w13 - l * ((-2/n) * sum(common_calc * w22_list))
        w14 = w14 - l * ((-2/n) * sum(common_calc * w22_list * x1_list))
        w15 = w15 - l * ((-2/n) * sum(common_calc * w22_list * x2_list))
    if a13:
        w16 = w16 - l * ((-2/n) * sum(common_calc * w23_list))
        w17 = w17 - l * ((-2/n) * sum(common_calc * w23_list * x1_list))
        w18 = w18 - l * ((-2/n) * sum(common_calc * w23_list * x2_list))
        
    i += 1
    

In [7]:
print('loss: ', loss)

loss:  0.0914538244813662


In [8]:
#calculating accuracy of our algorithm on train data
pred_result_train = []

for i in pred_result:
    if i < 0.5:
        pred_result_train.append(0)
    else:
        pred_result_train.append(1)

mat = confusion_matrix(train_y, pred_result_train)
print(mat)

print('\n')
score = accuracy_score(train_y, pred_result_train)
print(score)

[[185  14]
 [ 20 101]]


0.89375


In [9]:
#predicting on the test data with our algorithm

pred_result_test = []

for x1, x2 in test_x:    
    inputs = np.array([[x0],
                  [x1],
                  [x2]])
    out1 = np.dot(weights_1, inputs)

    z11 = out1[0,0]
    z12 = out1[1,0]
    z13 = out1[2,0]

    # activation relu
    a11 = np.maximum(0, z11)
    a12 = np.maximum(0, z12)
    a13 = np.maximum(0, z13)

    hidden_1 = np.array([a10, a11, a12, a13])

    weights_2 = np.array([[w20],
                   [w21],
                   [w22],
                   [w23]])

    out2 = np.dot(hidden_1, weights_2)
    z20 = out2[0]
    
    #activation sigmoid
    a20 = sigmoid(z20)
    
    if a20<0.5:
        pred_result_test.append(0)
    else:
        pred_result_test.append(1)  

In [10]:
#calculating accuracy of our model on test data
mat = confusion_matrix(test_y, pred_result_test)
print(mat)

print('\n')
score = accuracy_score(test_y, pred_result_test)
print(score)

[[55  3]
 [ 1 21]]


0.95
